In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
%matplotlib inline
import glob as gb
import tensorflow.keras as keras

#from tensorflow import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image

from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import backend as K

from IPython.display import SVG, Image
from livelossplot.tf_keras import PlotLossesCallback

from livelossplot import PlotLossesKeras
import tensorflow as tf
print("Tensorflow version:", tf.__version__)


Tensorflow version: 2.8.0


In [2]:
#!pip install utils

In [3]:
#!pip install tensorflow 1.15

In [4]:
#!pip install livelossplot

In [5]:
#!pip install keras

In [6]:
img_size = 48
batch_size = 64

datagen_train = ImageDataGenerator(horizontal_flip = True)
train_generator = datagen_train.flow_from_directory("train/",
                                                   target_size = (img_size, img_size),
                                                   color_mode = 'grayscale',
                                                   batch_size = batch_size,
                                                   class_mode = 'categorical',
                                                   shuffle = True)

datagen_validation = ImageDataGenerator(horizontal_flip = True)
validation_generator = datagen_train.flow_from_directory("test/",
                                                   target_size = (img_size, img_size),
                                                   color_mode = 'grayscale',
                                                   batch_size = batch_size,
                                                   class_mode = 'categorical',
                                                   shuffle = True)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [7]:
model = Sequential()

#1 - conv
model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#2 - conv
model.add(Conv2D(128, (5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#3 - conv
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#4 - conv layer
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

#1 - fully connected layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

#2 - fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

#last dense layer
model.add(Dense(7,activation = 'softmax'))

opt = Adam(lr=0.0005)
model.compile(optimizer=opt, loss ='categorical_crossentropy', metrics =['accuracy'])
model.summary()

#loss="sparse_categorical_crossentropy"

2022-08-03 15:20:00.875411: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       2

/Users/mac/opt/anaconda3/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [383]:
epochs = 100
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

checkpoint = ModelCheckpoint('model_weights.h5', monitor = 'val_accuracy',
                            save_weights_only=True, mode = 'max', verbose = 1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.1, patience = 2, min_lr=0.00001, model ='auto')

callbacks = [checkpoint, reduce_lr]

history = model.fit(
    x = train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = validation_steps,
    callbacks = callbacks
)

Epoch 1/40
448/448 [==============================] - ETA: 0s - loss: 1.8183 - accuracy: 0.3016
Epoch 1: saving model to model_weights.h5
448/448 [==============================] - 944s 2s/step - loss: 1.8183 - accuracy: 0.3016 - val_loss: 1.6168 - val_accuracy: 0.3744 - lr: 5.0000e-04
Epoch 2/40
448/448 [==============================] - ETA: 0s - loss: 1.4698 - accuracy: 0.4355
Epoch 2: saving model to model_weights.h5
448/448 [==============================] - 943s 2s/step - loss: 1.4698 - accuracy: 0.4355 - val_loss: 1.4490 - val_accuracy: 0.4495 - lr: 5.0000e-04
Epoch 3/40
448/448 [==============================] - ETA: 0s - loss: 1.3260 - accuracy: 0.4904 
Epoch 3: saving model to model_weights.h5
448/448 [==============================] - 4749s 11s/step - loss: 1.3260 - accuracy: 0.4904 - val_loss: 1.3434 - val_accuracy: 0.4845 - lr: 5.0000e-04
Epoch 4/40
448/448 [==============================] - ETA: 0s - loss: 1.2425 - accuracy: 0.5248
Epoch 4: saving model to model_weights.h

448/448 [==============================] - 904s 2s/step - loss: 0.8221 - accuracy: 0.6920 - val_loss: 0.9432 - val_accuracy: 0.6547 - lr: 1.0000e-05
Epoch 30/40
448/448 [==============================] - ETA: 0s - loss: 0.8244 - accuracy: 0.6925
Epoch 30: saving model to model_weights.h5
448/448 [==============================] - 905s 2s/step - loss: 0.8244 - accuracy: 0.6925 - val_loss: 0.9473 - val_accuracy: 0.6547 - lr: 1.0000e-05
Epoch 31/40
448/448 [==============================] - ETA: 0s - loss: 0.8200 - accuracy: 0.6949
Epoch 31: saving model to model_weights.h5
448/448 [==============================] - 900s 2s/step - loss: 0.8200 - accuracy: 0.6949 - val_loss: 0.9491 - val_accuracy: 0.6546 - lr: 1.0000e-05
Epoch 32/40
448/448 [==============================] - ETA: 0s - loss: 0.8182 - accuracy: 0.6915
Epoch 32: saving model to model_weights.h5
448/448 [==============================] - 898s 2s/step - loss: 0.8182 - accuracy: 0.6915 - val_loss: 0.9438 - val_accuracy: 0.6542 -

In [6]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

NameError: name 'model' is not defined

In [384]:
model.save("model.h5")

In [14]:
model.save("export.pkl")

INFO:tensorflow:Assets written to: export.pkl/assets


In [15]:
python -m ipykernel install --user --name=fastai


SyntaxError: invalid syntax (3977759552.py, line 1)

In [9]:
from tensorflow import keras


In [10]:
from tensorflow.keras.models import model_from_json

In [11]:
from __future__ import division

In [12]:
import os

In [13]:
import cv2
from PIL import Image

In [13]:
json_file = open('model.json', 'r')

In [14]:
loaded_model_json = json_file.read()

In [15]:
json_file.close()

In [18]:
loaded_model = model_from_json(loaded_model_json)

In [19]:
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [20]:
#setting image resizing parameters
WIDTH = 48
HEIGHT = 48
x=None
y=None
labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [31]:
#loading image
full_size_image = cv2.imread("sad2.jpeg")
print("Image Loaded")


Image Loaded


In [32]:
gray=cv2.cvtColor(full_size_image,cv2.COLOR_BGR2GRAY)
face = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
faces = face.detectMultiScale(gray, 1.3  , 3)

In [33]:
#print("voici",gray)

In [34]:
#faces = face.detectMultiScale(
    #gray,
    #scaleFactor=1.3,
    #minNeighbors=5,
    #minSize=(30, 30),
    #flags = cv2.CASCADE_SCALE_IMAGE)
 
#print("Faces shape : ", faces.shape[0])


In [35]:
print(type(faces), faces)

<class 'numpy.ndarray'> [[238  94 166 166]]


In [49]:
#for (x,y,w,h) in faces:
    #cv2.rectangle(full_size_image,(x,y),(x+w,y+h),(255,0,0),2)
 
#print("Face count : ", faces.shape[0])
 


In [50]:
#cv2.imshow('img',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [36]:
#detecting faces
for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
        cv2.rectangle(full_size_image, (x, y), (x + w, y + h), (0, 255, 0), 1)
        
        #predicting the emotion
        yhat= loaded_model.predict(cropped_img)
        cv2.putText(full_size_image, labels[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
        print("Emotion: "+labels[int(np.argmax(yhat))])



Emotion: sad


In [37]:
cv2.imshow('Emotion', full_size_image)
cv2.waitKey(10)
cv2.destroyAllWindows()

In [420]:
#loaded_model = tf.keras.models.load_model('model.h5')

In [412]:
#!pip install opencv-python

In [421]:
#import cv2
#from PIL import Image

In [441]:
#image = cv2.imread("sadder.png")

In [442]:
#gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [443]:
#image_fromarray = Image.fromarray(gray,'L')

In [444]:
#resize_image = image_fromarray.resize((48, 48))
#expand_input = np.expand_dims(resize_image,axis=0)
#input_data = np.array(expand_input)
#input_data = input_data/255

In [445]:
#input_data.shape

(1, 48, 48)

In [446]:
#pred = loaded_model.predict(input_data)
#result = pred.argmax()

In [450]:
#result

0

In [449]:
#print(result.shape)

()


In [430]:
#image_fromarray = Image.fromarray(image, 'RGB')


#img = Image.open('surprise.jpeg').convert('L')
#img.save('surprise.jpeg')
#img = Image.fromarray(img, 'grayscale')

In [431]:
#img

In [392]:
#image = cv2.imread("sad.jpg")

In [432]:
#resize_image = img.resize((48, 48))

In [433]:
#expand_input = np.expand_dims(resize_image,axis=0)

In [434]:
#input_data = np.array(expand_input)

In [435]:
#input_data = input_data/255

In [436]:
#input_data.shape

In [437]:
#predd = loaded_model.predict(input_data)

In [438]:
#result = predd.argmax()

In [439]:
#result

In [401]:
#train_generator.class_indices


{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [21]:
class_indices

NameError: name 'class_indices' is not defined